<a href="https://colab.research.google.com/github/Quenassier/traintest2EASY/blob/main/traintest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

In [10]:
# Чтение данных обучения и теста из CSV файлов
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [11]:
# Замена отсутствующих значений в возрасте средним значением
imputer = SimpleImputer(strategy="mean")
train_data["Age"] = imputer.fit_transform(train_data[["Age"]])
test_data["Age"] = imputer.transform(test_data[["Age"]])

In [12]:
# Замена отсутствующих значений в столбце "Embarked" на самое часто встречающееся значение
train_data["Embarked"].fillna(train_data["Embarked"].mode()[0], inplace=True)
test_data["Embarked"].fillna(test_data["Embarked"].mode()[0], inplace=True)

In [13]:
# Замена отсутствующих значений в столбце "Fare" на среднее значение
test_data["Fare"].fillna(test_data["Fare"].mean(), inplace=True)

In [14]:
# Преобразование категориальных признаков в числовые с помощью LabelEncoder
le = LabelEncoder()
train_data["Sex"] = le.fit_transform(train_data["Sex"])
test_data["Sex"] = le.transform(test_data["Sex"])
train_data["Embarked"] = le.fit_transform(train_data["Embarked"])
test_data["Embarked"] = le.transform(test_data["Embarked"])

In [15]:

# Выделение признаков и целевой переменной для обучения модели логистической регрессии
X_train = train_data[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
y_train = train_data["Survived"]

In [16]:
# Обучение модели логистической регрессии
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [17]:
# Замена отсутствующих значений в возрасте средним значением и обучение модели линейной регрессии для предсказания возраста
train_data["Age"].fillna(train_data["Age"].mean(), inplace=True)
X_age_train = train_data[["Pclass", "Sex", "SibSp", "Parch", "Fare", "Embarked"]]
y_age_train = train_data["Age"]
# линейная регрессия
linreg = LinearRegression()
linreg.fit(X_age_train, y_age_train)

LinearRegression()

In [18]:
# Вывод коэффициентов и свободного члена модели линейной регрессии
print("Coefficients:", linreg.coef_)
print("Intercept:", linreg.intercept_)

Coefficients: [-5.50407422  2.26374912 -1.83902977 -1.29090714 -0.01035394  0.43609156]
Intercept: 42.05790541903572


In [19]:
# Обучение модели k-ближайших соседей для предсказания значения Embarked
X_embarked_train = train_data[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]]
y_embarked_train = train_data["Embarked"]
knn = KNeighborsClassifier()
knn.fit(X_embarked_train, y_embarked_train)

KNeighborsClassifier()

In [20]:
# Преобразование категориальных признаков в числовые с помощью LabelEncoder для тестовых данных
test_data_copy = test_data.copy()
label_encoder = LabelEncoder()
test_data["Sex"] = label_encoder.fit_transform(test_data["Sex"])
test_data["Embarked"] = label_encoder.fit_transform(test_data["Embarked"])

In [21]:
# Импьютация отсутствующих значений средним значением для тестовых данных
imputer = SimpleImputer(strategy="mean")
columns_to_impute = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
test_data[columns_to_impute] = imputer.fit_transform(test_data[columns_to_impute])

In [22]:
# Обучение модели линейной регрессии для предсказания возраста
linreg.fit(X_train, y_train)
y_age_pred = linreg.predict(test_data[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]])

In [23]:
# Заполнение предсказанными значениями возраста и сохранение результатов в CSV файл
test_data["Age"] = y_age_pred
test_data.to_csv("predicted_test.csv", index=False)

In [24]:
# Подготовка данных для предсказания выживания
X_test = test_data[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]

In [25]:
# Предсказание выживания с помощью модели логистической регрессии
y_survived_pred = logreg.predict(X_test)

In [26]:
# Предсказание возраста с помощью модели линейной регрессии
y_age_pred = linreg.predict(X_test[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]])


In [27]:
# Предсказание значения Embarked с помощью модели k-ближайших соседей
y_embarked_pred = knn.predict(X_test[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]])

In [28]:

# Обновление данных теста с предсказанными значениями и сохранение результатов в файл
test_data["Survived"] = y_survived_pred
test_data["Age"] = y_age_pred
test_data["Embarked"] = y_embarked_pred
test_data.to_csv("finally_test.csv", index=False)

In [29]:
print(train_data)
print(test_data)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name  Sex        Age  SibSp  \
0                              Braund, Mr. Owen Harris    1  22.000000      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...    0  38.000000      1   
2                               Heikkinen, Miss. Laina    0  26.000000      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)    0  35.000000      1   
4                             Allen, Mr. William Henry    1  35.000000      0   
..                                                 ...  .

In [30]:
# Подготовка данных для предсказания
X_test = test_data[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]].copy()
print(X_test.columns)

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')


In [31]:
print(train_data)
print(test_data)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name  Sex        Age  SibSp  \
0                              Braund, Mr. Owen Harris    1  22.000000      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...    0  38.000000      1   
2                               Heikkinen, Miss. Laina    0  26.000000      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)    0  35.000000      1   
4                             Allen, Mr. William Henry    1  35.000000      0   
..                                                 ...  .